# Define Rules and Policies

## Authorisation

In [1]:
import json
import requests # type: ignore
import time

LOCAL_DIR_PREFIX = ""

# uncomment the next line if you cloned the repository and the notebook will run locally on your laptop
# LOCAL_DIR_PREFIX = "../../"

# uncomment the next line if hardcoding the credentials on the code, global class variables
FILE_CREDENTIALS = ""

# uncomment the next two lines if using the file ikcapikey.json for storing credentials
# FILE_CREDENTIALS = "python/ikcapikey.json"
# FILE_CREDENTIALS = LOCAL_DIR_PREFIX + FILE_CREDENTIALS

class credentials :

    file_credentials = ""
    url_server = " https://cpd-cpd.apps.6645c6d6ca5b92001e29286f.cloud.techzone.ibm.com"
    username = "admin"
    apikey = "SfpLD0yMQFh4xpdOrgPuTK9AdBtEVEqF1gK2HSlw"
    access_token = ""

    def __init__(self, file_credentials):

        if file_credentials != "" :
            try :
                with open(file_credentials) as f :
                    data = json.load(f)
                    self.url_server = data["url_server"]
                    self.username = data["username"]
                    self.apikey = data["api_key"]
                    self.file_credentials = file_credentials
            except :
                print("Error with the file ", file_credentials)
    
   
    def urlRequest(self, urlSuffix):
        return self.url_server + urlSuffix

    def get_bearer_token(self):
        
        # Get a bearer token with the API key - Cloud Pak for Data SaaS
        # url = "https://iam.cloud.ibm.com/identity/token"
        # headers = {"Content-Type" : "application/x-www-form-urlencoded"}
        # data = "grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey=" + apikey
        # r = requests.post(url, headers=headers, data=data)
        # access_token = r.json()["access_token"]

        # Get a bearer token with the API key - Cloud Pak for Data Software
        urlSuffix = "/icp4d-api/v1/authorize"
        headers = {'Accept': 'application/json', 'Content-type': 'application/json'}
        data = {"username" : self.username, "api_key" : self.apikey}
        r = requests.post(self.urlRequest(urlSuffix), headers=headers, data=json.dumps(data))

        if r.status_code != 200:
            print("Error with the request. Code: ", r.status_code)
            print(r.text)
        else :
            try:
                self.access_token = r.json()["token"]
            except KeyError:
                print("Error with the token. Code: ", r.status_code)
                print("Hint: check the credentials file ", self.file_credentials)
                print(r.text)
                
            return self.access_token

myconn = credentials(FILE_CREDENTIALS)
access_token = myconn.get_bearer_token()


## Define Rules and Policies


### 1. Create Governance Rules

1.a. Add new rules

In [ ]:
print("---- Import Governance Rules from CSV----")

IMPORT_CSV_FILE = "artifacts/governance-rules.csv"
IMPORT_CSV_FILE = LOCAL_DIR_PREFIX + IMPORT_CSV_FILE

urlSuffix='/v3/governance_artifact_types/rule/import?merge_option=all'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
files = {'file': (IMPORT_CSV_FILE, open(IMPORT_CSV_FILE, 'rb'), 'text/csv')}

r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, files=files)

if r.status_code == 200 :
    status = r.json()["status"]
    print("Import finished. Status = ", status)
    print(r.text)
elif r.status_code == 202 :
    process_id = r.json()["process_id"]
    print(f"----- Import process started: {process_id} ----- ")
    print("----- Entering wait loop ------")
    urlSuffix='/v3/governance_artifact_types/import/status/' + process_id
    headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
    while True :
        r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)
        if r.status_code != 200 :
            print("Error with the request. Code: ", r.status_code)
            print(r.text)
        status = r.json()["status"]
        if status != "IN_PROGRESS" :
            break
        else :
            print ("Import in progess, please wait")
            time.sleep(5)
else :
    print("Error with the request. Code: ", r.status_code)
    print(r.text)

workflow_id = r.json()["workflow_id"]


#### 1.b. Publish the changes

In [ ]:
urlSuffix='/v3/workflows/' + workflow_id + '?include_user_tasks=true'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)

user_tasks = r.json()["entity"]["user_tasks"]
for i in user_tasks :
    if i["metadata"]["workflow_id"] == workflow_id :
        task_id = i["metadata"]["task_id"]

urlSuffix='/v3/workflow_user_tasks/' + task_id + '/actions'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
payload = {'action': 'complete', 'form_properties': [{'id': 'action', 'value': '#publish'}]}
r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, json=payload)

if r.status_code == 202 or r.status_code == 204 :
    print("Publish Successful, Code = ", r.status_code)
else :
    print("Error in publishing artifacts, Code = ", r.status_code)
    print(r.text)

### 2. Create Email Protection Rule

In [ ]:
# Look for the global id of the data class called "Email Address"

urlSuffix = "/v3/search"
headers = {"content-type" : "application/json", "Authorization" : "Bearer " + access_token}
data_class_name = "Email Address"
data_class_name_for_search = "Email ?Address" # The lucene syntax requires a ? after the space
data={
    "_source":[ "metadata.name", "entity.artifacts.global_id"],
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "metadata.artifact_type": "data_class"
                    }
                },
                {
                    "match": {
                        "metadata.name" : data_class_name_for_search
                    }
                }
            ]
        }
    }
}
r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, json=data)

if r.status_code != 200 :
    print("Error with the request. Code: ", r.status_code)
    print(r.text)

else : 

    for i in r.json()["rows"] :  # Never trust the lucene syntax... who knows what it retrieves
        if i["metadata"]["name"] == data_class_name :
            global_id = i["entity"]["artifacts"]["global_id"]

    # With this global id, create the rule

    urlSuffix='/v3/enforcement/rules'
    headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
    data = {
                "name": "Protect Email Address",
                "description": "Protect all email addresses using the data privacy advanced masking method.",
                "governance_type_id": "Access",
                "trigger": [
                    "$Asset.InferredClassification",
                    "CONTAINS",
                    [
                        '$' + global_id
                    ]
                ],
                "action": {
                    "name": "Transform",
                    "subaction": {
                        "name": "pseudonymizeTerms",
                        "parameters": [
                            {
                                "name": "term_name",
                                "value": global_id
                            },
                            {
                                "name": "maskingType",
                                "value": "Partial"
                            },
                            {
                                "name": "maskingProcessor",
                                "value": "RepeatableFormatFabrication"
                            },
                            {
                                "name": "preserveFormat",
                                "value": "true"
                            },
                            {
                                "name": "maskingOptions",
                                "value": [
                                    {
                                        "name": "User name",
                                        "value": "Generate user name"
                                    },
                                    {
                                        "name": "Domain name",
                                        "value": "Original"
                                    }
                                ]
                            }
                        ]
                    }
                },
                "state": "active"
            }

    r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, json=data)
    
    if r.status_code not in (200, 201) :
        print("Error with the request. Code: ", r.status_code)
        print(r.text)


### 3. Create Phone Protection Rule

In [ ]:
# Look for the global id of the data class called "Phone Number"

urlSuffix = "/v3/search"
headers = {"content-type" : "application/json", "Authorization" : "Bearer " + access_token}
data_class_name = "Phone Number"
data_class_name_for_search = "Phone ?Number" # The lucene syntax requires a ? after the space
data={
    "_source":[ "metadata.name", "entity.artifacts.global_id"],
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "metadata.artifact_type": "data_class"
                    }
                },
                {
                    "match": {
                        "metadata.name" : data_class_name_for_search
                    }
                }
            ]
        }
    }
}
r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, json=data)

if r.status_code != 200 :
    print("Error with the request. Code: ", r.status_code)
    print(r.text)
else : 
    for i in r.json()["rows"] :  # Never trust the lucene syntax... who knows what it retrieves
        if i["metadata"]["name"] == data_class_name :
            global_id = i["entity"]["artifacts"]["global_id"]

    # With this global id, create the rule

    urlSuffix='/v3/enforcement/rules'
    headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
    data = {            
            "name": "Protect Phone Numbers",
            "description": "Protect all phone numbers using the redaction data privacy masking method.",
            "governance_type_id": "Access",
            "trigger": [
                "$Asset.InferredClassification",
                "CONTAINS",
                [
                    '$' + global_id
                ]
            ],
            "action": {
                "name": "Transform",
                "subaction": {
                    "name": "redactTerms",
                    "parameters": [
                        {
                            "name": "term_name",
                            "value": global_id
                        },
                        {
                            "name": "maskingChar",
                            "value": "X"
                        },
                        {
                            "name": "maskingType",
                            "value": "Full"
                        },
                        {
                            "name": "preserveFormat",
                            "value": "true"
                        }
                    ]
                }
            },
            "state": "active"
        }

    r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, json=data)
    
    if r.status_code not in (200, 201) :
        print("Error with the request. Code: ", r.status_code)
        print(r.text)


### 4. Create US SSN Protection Rule

In [ ]:
# Look for the global id of the data class called "Phone Number"

urlSuffix = "/v3/search"
headers = {"content-type" : "application/json", "Authorization" : "Bearer " + access_token}
data_class_name = "US Social Security Number"
data_class_name_for_search = "US ?Social ?Security ?Number" # The lucene syntax requires a ? after the space
data={
    "_source":[ "metadata.name", "entity.artifacts.global_id"],
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "metadata.artifact_type": "data_class"
                    }
                },
                {
                    "match": {
                        "metadata.name" : data_class_name_for_search
                    }
                }
            ]
        }
    }
}
r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, json=data)

if r.status_code != 200 :
    print("Error with the request. Code: ", r.status_code)
    print(r.text)
else : 
    for i in r.json()["rows"] :  # Never trust the lucene syntax... who knows what it retrieves
        if i["metadata"]["name"] == data_class_name :
            global_id = i["entity"]["artifacts"]["global_id"]

    # With this global id, create the rule

    urlSuffix='/v3/enforcement/rules'
    headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
    data = {            
           "name": "Protect US Social Security Numbers",
            "description": "Protect all US social security numbers using the data privacy advanced masking method.",
            "governance_type_id": "Access",
            "trigger": [
                "$Asset.InferredClassification",
                "CONTAINS",
                [
                    "$" +  global_id
                ]
            ],
            "action": {
                "name": "Transform",
                "subaction": {
                    "name": "pseudonymizeTerms",
                    "parameters": [
                        {
                            "name": "term_name",
                            "value": global_id
                        },
                        {
                            "name": "maskingType",
                            "value": "Full"
                        },
                        {
                            "name": "maskingProcessor",
                            "value": "FormatPreservingTokenization"
                        },
                        {
                            "name": "preserveFormat",
                            "value": "true"
                        }
                    ]
                }
            },
            "state": "active"
        }

    r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, json=data)
    
    if r.status_code not in (200, 201) :
        print("Error with the request. Code: ", r.status_code)
        print(r.text)


### 5. Create Policies

#### 5.a. Add new Policies

In [ ]:
print("---- Import Policies from CSV----")

IMPORT_CSV_FILE = "artifacts/governance-policies.csv"
IMPORT_CSV_FILE = LOCAL_DIR_PREFIX + IMPORT_CSV_FILE

urlSuffix='/v3/governance_artifact_types/policy/import?merge_option=all'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
files = {'file': (IMPORT_CSV_FILE, open(IMPORT_CSV_FILE, 'rb'), 'text/csv')}

r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, files=files)

if r.status_code == 200 :
    status = r.json()["status"]
    print("Import finished. Status = ", status)
    print(r.text)
elif r.status_code == 202 :
    process_id = r.json()["process_id"]
    print(f"----- Import process started: {process_id} ----- ")
    print("----- Entering wait loop ------")
    urlSuffix='/v3/governance_artifact_types/import/status/' + process_id
    headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
    while True :
        r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)
        if r.status_code != 200 :
            print("Error with the request. Code: ", r.status_code)
            print(r.text)
        status = r.json()["status"]
        if status != "IN_PROGRESS" :
            break
        else :
            print ("Import in progess, please wait")
            time.sleep(5)
else :
    print("Error with the request. Code: ", r.status_code)
    print(r.text)

workflow_id = r.json()["workflow_id"]


#### 5.b. Publish the changes

In [ ]:
urlSuffix='/v3/workflows/' + workflow_id + '?include_user_tasks=true'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)

user_tasks = r.json()["entity"]["user_tasks"]
for i in user_tasks :
    if i["metadata"]["workflow_id"] == workflow_id :
        task_id = i["metadata"]["task_id"]

urlSuffix='/v3/workflow_user_tasks/' + task_id + '/actions'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
payload = {'action': 'complete', 'form_properties': [{'id': 'action', 'value': '#publish'}]}
r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, json=payload)

if r.status_code == 202 or r.status_code == 204 :
    print("Publish Successful, Code = ", r.status_code)
else :
    print("Error in publishing artifacts, Code = ", r.status_code)
    print(r.text)